In [2]:
import glob
import os
import csv

import pandas as pd
import flow_package as fp

In [3]:
def get_csv_files(dir_name_list):
    for _dir in dir_name_list:
        paths = glob.glob(f"./raw/{_dir[0]}/*.csv")
        
        for path in paths:
            print(path)
            filename = os.path.basename(path)
            read_write(path, f"./raw_after/{_dir[0]}/{filename}", _dir[2])


def read_write(path, write_path, labels=None):
    with open(path, "r", encoding="utf-8") as inputs, open(write_path, "w", newline="") as outputs:
        reader = csv.reader(inputs)
        writer = csv.writer(outputs)

        headers = next(reader)
        if not labels:
            headers = [ head.strip() for head in headers ]
            writer.writerow(headers)
        else:
            writer.writerow(labels)

        for row in reader:
            if row == headers:
                break
            writer.writerow(row)

# CSV Preprocessing

## data path

Please save `./raw/...`

```text
# example

raw
    cicids2018
        ...
```

## index rewrite

Please rewrite column name in flow_package.const.__feature_labels

If not all correct, adjust name.

```python
get_csv_files(
    [
        [
            "cicids2018",
            "BENIGN",
            [
                # feature label list
            ]
        ]
    ]
)
```

In [4]:
datasets = [
    ["cicids2017", "BENIGN", None],
    [
        "csecicids2018",
        "Benign",
        ["Destination Port","Protocol","Timestamp","Flow Duration","Total Fwd Packets","Total Backward Packets","Total Length of Fwd Packets","Total Length of Bwd Packets","Fwd Packet Length Max","Fwd Packet Length Min","Fwd Packet Length Mean","Fwd Packet Length Std","Bwd Packet Length Max","Bwd Packet Length Min","Bwd Packet Length Mean","Bwd Packet Length Std","Flow Bytes/s","Flow Packets/s","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Total","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max","Fwd IAT Min","Bwd IAT Total","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags","Fwd Header Length","Bwd Header Length","Fwd Packets/s","Bwd Packets/s","Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Std","Packet Length Variance","FIN Flag Count","SYN Flag Count","RST Flag Count","PSH Flag Count","ACK Flag Count","URG Flag Count","CWE Flag Count","ECE Flag Count","Down/Up Ratio","Average Packet Size","Avg Fwd Segment Size","Avg Bwd Segment Size","Fwd Avg Bytes/Bulk","Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate","Subflow Fwd Packets","Subflow Fwd Bytes","Subflow Bwd Packets","Subflow Bwd Bytes","Init_Win_bytes_forward","Init_Win_bytes_backward","act_data_pkt_fwd","min_seg_size_forward","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min","Label"]
    ]
]

get_csv_files(datasets)

./raw/cicids2017\Benign-Monday-WorkingHours.pcap_ISCX.csv
./raw/cicids2017\Botnet-Friday-WorkingHours-Morning.pcap_ISCX.csv
./raw/cicids2017\Bruteforce-Tuesday-WorkingHours.pcap_ISCX.csv
./raw/cicids2017\DDoS-Friday-WorkingHours-Afternoon.pcap_ISCX.csv
./raw/cicids2017\DoS-Wednesday-WorkingHours.pcap_ISCX.csv
./raw/cicids2017\Infiltration-Thursday-WorkingHours-Afternoon.pcap_ISCX.csv
./raw/cicids2017\Portscan-Friday-WorkingHours-Afternoon.pcap_ISCX.csv
./raw/cicids2017\WebAttacks-Thursday-WorkingHours-Morning.pcap_ISCX.csv
./raw/csecicids2018\02-14-2018.csv
./raw/csecicids2018\02-15-2018.csv
./raw/csecicids2018\02-16-2018.csv
./raw/csecicids2018\02-20-2018.csv
./raw/csecicids2018\02-21-2018.csv
./raw/csecicids2018\02-22-2018.csv
./raw/csecicids2018\02-23-2018.csv
./raw/csecicids2018\02-28-2018.csv
./raw/csecicids2018\03-01-2018.csv
./raw/csecicids2018\03-02-2018.csv


In [5]:
def collect_unique_data(dir_name_list):
    for dir_name in dir_name_list:
        subset_list = ["Protocol", "Label"]
        paths = glob.glob(f"./raw_after/{dir_name[0]}/*.csv")

        result_list = [pd.DataFrame() for _ in range(2)]
        for path in paths:
            df_array = unique_dataframe(path)
            for i in range(2):
                result_list[i] = pd.concat([result_list[i], df_array[i]], axis=0)
        else:
            df = pd.DataFrame()
            for result, subset_label in zip(result_list, subset_list):
                df = pd.concat([df, result.drop_duplicates(subset=[subset_label])], axis=0)
            print(len(df))
            df.drop_duplicates().to_csv(f"./raw_after/unique/{dir_name[0]}.csv")


def unique_dataframe(path):
    print(path, end=" ")
    df = pd.read_csv(path)
    df_protocol = df.drop_duplicates(subset=["Protocol"])
    df_label = df.drop_duplicates(subset=["Label"])
    print(f"protocol: {len(df_protocol)}, label: {len(df_label)}")

    return [df_protocol, df_label]

## category data

For One-Hot Encoding, collect unique data.

In [6]:
collect_unique_data(datasets)

./raw_after/cicids2017\Benign-Monday-WorkingHours.pcap_ISCX.csv protocol: 3, label: 1
./raw_after/cicids2017\Botnet-Friday-WorkingHours-Morning.pcap_ISCX.csv protocol: 3, label: 2
./raw_after/cicids2017\Bruteforce-Tuesday-WorkingHours.pcap_ISCX.csv protocol: 3, label: 3
./raw_after/cicids2017\DDoS-Friday-WorkingHours-Afternoon.pcap_ISCX.csv protocol: 3, label: 2
./raw_after/cicids2017\DoS-Wednesday-WorkingHours.pcap_ISCX.csv protocol: 3, label: 6
./raw_after/cicids2017\Infiltration-Thursday-WorkingHours-Afternoon.pcap_ISCX.csv protocol: 3, label: 2
./raw_after/cicids2017\Portscan-Friday-WorkingHours-Afternoon.pcap_ISCX.csv protocol: 3, label: 2
./raw_after/cicids2017\WebAttacks-Thursday-WorkingHours-Morning.pcap_ISCX.csv protocol: 3, label: 4
18
./raw_after/csecicids2018\02-14-2018.csv protocol: 3, label: 3
./raw_after/csecicids2018\02-15-2018.csv protocol: 3, label: 3
./raw_after/csecicids2018\02-16-2018.csv protocol: 3, label: 3
./raw_after/csecicids2018\02-20-2018.csv protocol: 3, l

In [9]:
def filtered_data(lists):
    N = 50000


    for dir_name in lists:
        csv_dtype = {
            "Protocol": "category",
            "Label": "category",
        }
        print(dir_name[0])
        paths = glob.glob(f"./raw_after/{dir_name[0]}/*.csv")

        base = pd.read_csv(
            f"./raw_after/unique/{dir_name[0]}.csv",
            dtype=csv_dtype,
        )
        before_list = []

        for path in paths:
            print(path, end=" ")
            df = pd.read_csv(
                path,
                dtype=csv_dtype,
            )
            basename = os.path.basename(path)
            df = pd.concat([base, df], axis=0)
            df, label_list, ohe_labels = fp.preprocessing_until_all(df, categorical_index=["Protocol"])
            if len(before_list) == 0:
                before_list = label_list
            else:
                for before, after in zip(before_list, label_list):
                    if before != after:
                        raise ValueError(f"label_list={label_list} != before_list={before_list}")
                # if before_list != label_list:
                #     raise ValueError(f"label_list={label_list} != before_list={before_list}")
            df = df[len(base):]
            print(f"size={len(df)}")
            if len(df) < N:
                print(f"write...")
                df.to_csv(
                    f"./raw_after_filtered/{dir_name[0]}/data/{basename}",
                )
            else:
                for size in range(0, len(df), N):
                    print(f"{size} --> ", end="")
                    end = size + N if size + N < len(df) else len(df)
                    df.iloc[size:end].to_csv(
                        f"./raw_after_filtered/{dir_name[0]}/data/{basename}_{size//N}.csv",
                    )
                    # print(f"{size + N} --> ", end="")
                else:
                    print()
            with open(f"./raw_after_filtered/{dir_name[0]}/label_list.txt", "w") as f:
                for label in label_list:
                    f.write(f"{label}\n")
            with open(f"./raw_after_filtered/{dir_name[0]}/ohe_labels.txt", "w") as f:
                for label in ohe_labels:
                    f.write(f"{label}\n")

## Last

adjust data

In [10]:
filtered_data(datasets)

cicids2017
./raw_after/cicids2017\Benign-Monday-WorkingHours.pcap_ISCX.csv === done
size=529918
0 --> 50000 --> 100000 --> 150000 --> 200000 --> 250000 --> 300000 --> 350000 --> 400000 --> 450000 --> 500000 --> 
./raw_after/cicids2017\Botnet-Friday-WorkingHours-Morning.pcap_ISCX.csv === done
size=191033
0 --> 50000 --> 100000 --> 150000 --> 
./raw_after/cicids2017\Bruteforce-Tuesday-WorkingHours.pcap_ISCX.csv === done
size=445909
0 --> 50000 --> 100000 --> 150000 --> 200000 --> 250000 --> 300000 --> 350000 --> 400000 --> 
./raw_after/cicids2017\DDoS-Friday-WorkingHours-Afternoon.pcap_ISCX.csv === done
size=225745
0 --> 50000 --> 100000 --> 150000 --> 200000 --> 
./raw_after/cicids2017\DoS-Wednesday-WorkingHours.pcap_ISCX.csv === done
size=692703
0 --> 50000 --> 100000 --> 150000 --> 200000 --> 250000 --> 300000 --> 350000 --> 400000 --> 450000 --> 500000 --> 550000 --> 600000 --> 650000 --> 
./raw_after/cicids2017\Infiltration-Thursday-WorkingHours-Afternoon.pcap_ISCX.csv === done
size